**Laboratorio N°5 - SIS420**

**Introducción.**

En este cuadernillo, para este laboratorio, nos enfocaremos en el entrenamiento y evaluación de redes neuronales, la primera parte tendrá la implementación manual de una red neuronal y la segunda parte tendrá la implementación de una red neuronal con PyTorch.

**Objetivos.**

- Preparar el dataset, usando pandas, para el entrenamiento correcto de la red neuronal.
- Implementar el código respectivo para la creación de una red neuronal (manual y PyTorch).
- Entrenar la red neuronal para la realización de predicciones (manual y PyTorch).
- Evaluar la efectividad de las predicciones de la red neuronal (manual y PyTorch).
- Establecer conclusiones y comparaciones respecto a las implementaciones.

**(0) Declaración de Librerías.** Antes que nada, declararemos las respectivas librerías que se usarán en este cuadernillo, así evitaremos redundancias y se tendrá un código más ordenado y limpio. Además se montará el drive para trabajar con los archivos de Google Drive.

**Importación de bibliotecas:**
- import os: Importa la biblioteca os, que proporciona funciones para interactuar con el sistema operativo, como manipulación de archivos y directorios.
- import numpy as np: Importa la biblioteca numpy y la renombra como np. numpy es ampliamente utilizado para cálculos numéricos y manipulación de matrices.
- import pandas as pd: Importa la biblioteca pandas y la renombra como pd. pandas se utiliza para el análisis y manipulación de datos tabulares.
- from matplotlib import pyplot as plt: Importa la función pyplot de la biblioteca matplotlib y la renombra como plt.
- matplotlib se utiliza para crear gráficos y visualizaciones.
- from scipy import optimize: Importa la biblioteca scipy y su módulo optimize. scipy proporciona herramientas para optimización y análisis numérico.
- from sklearn.model_selection import train_test_split: Importa la función train_test_split de la biblioteca sklearn. Se utiliza para dividir conjuntos de datos en conjuntos de entrenamiento y prueba.
- import torch: Importa la biblioteca torch, que se utiliza para trabajar con redes neuronales y tensores.
- from torch import optim: Importa el módulo optim de la biblioteca torch. Proporciona algoritmos de optimización para entrenar modelos de aprendizaje profundo.
- from torch import nn: Importa el módulo nn de la biblioteca torch. Contiene clases para construir redes neuronales.
- from torch.utils.data import TensorDataset, DataLoader: Importa las clases TensorDataset y DataLoader de la biblioteca torch.utils.data. Se utilizan para cargar datos en tensores y crear lotes para el entrenamiento.
- import torchvision: Importa la biblioteca torchvision, que proporciona conjuntos de datos y transformaciones para la visión por computadora.
- import torchvision.transforms as transforms: Importa el módulo transforms de torchvision. Se utiliza para aplicar transformaciones a imágenes.
- import torch.nn.functional as F: Importa el módulo F de la biblioteca torch.nn. Contiene funciones de activación y otras operaciones.
- import torchvision.datasets as datasets: Importa el módulo datasets de torchvision. Proporciona conjuntos de datos estándar para la visión por computadora.
- from tqdm import tqdm: Importa la función tqdm de la biblioteca tqdm. Se utiliza para mostrar barras de progreso en bucles.
- from google.colab import drive: Importa la función drive de la biblioteca google.colab. Se utiliza para montar Google Drive en Google Colab.
- %matplotlib inline: Configura la visualización de gráficos en línea en el entorno de Google Colab.
**Montaje de Google Drive:**
La línea drive.mount('/content/gdrive') monta Google Drive en la ubicación /content/gdrive. Esto permite acceder a archivos almacenados en Google Drive desde el entorno de Colab.

In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import optimize
from sklearn.model_selection import train_test_split
import torch
from torch import optim
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.datasets as datasets
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/gdrive')
%matplotlib inline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Primera Parte**


**(1) Declaración de las funciones.**

La función *cargarDataset* se encarga de cargar y preparar un conjunto de datos para el entrenamiento y la evaluación de un modelo de Inteligencia Artificial.

In [2]:
def cargarDataset(_dataset):
  dataset = preprocesarDataset(_dataset)
  train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=99)
  X_train, y_train = train_set.iloc[:, :-1], train_set.iloc[:, -1]
  X_test, y_test = test_set.iloc[:, :-1], test_set.iloc[:, -1]
  X_train = np.concatenate([np.ones((X_train.shape[0], 1)), X_train], axis=1)
  X_test = np.concatenate([np.ones((X_test.shape[0], 1)), X_test], axis=1)
  return X_train, X_test, y_train, y_test

La función *preprocesarDataset* se utiliza para preparar un conjunto de datos antes de utilizarlo en un modelo de inteligencia artificial.
- Carga el conjunto de datos desde un archivo CSV.
- Identifica y procesa las columnas con datos categóricos, convirtiéndolos en valores numéricos para que puedan ser utilizados.
- Maneja los valores faltantes reemplazándolos con ceros.
- Mantiene la columna de etiquetas (‘labels’) al final del conjunto de datos para su uso posterior.

In [3]:
def preprocesarDataset(_dataset):
  dataset = pd.read_csv(_dataset, sep=',', header=0, decimal='.')
  datos = {}
  columnas = dataset.columns[dataset.dtypes == 'object'].tolist()
  for columna in columnas:
    datos[columna] = dataset[columna].drop_duplicates().values
  datos_num = {}
  for columna, valores in datos.items():
    indice_reemp = 0
    datos_num_col = {}
    for valor in valores:
      if valor not in datos_num_col and not pd.isnull(valor):
        datos_num_col[valor] = indice_reemp
        indice_reemp += 1
    if np.nan not in datos_num_col:
      datos_num_col[np.nan] = 0
    datos_num[columna] = datos_num_col
  for columna, d_n in datos_num.items():
    dataset[columna] = dataset[columna].replace(d_n)
  dataset = dataset.fillna(0)
  etiquetas = dataset.pop('labels')
  dataset.insert(len(dataset.columns), 'labels', etiquetas)
  return dataset

La función *normalizarCaracteristicas* se utiliza para normalizar las características de un conjunto de datos.
La normalización es un paso importante en el preprocesamiento de datos para muchos algoritmos de machine learning, ya que pone todas las características en una escala común sin distorsionar las diferencias en los rangos de valores. Además, la normalización puede mejorar la convergencia de algoritmos de optimización.

In [4]:
def normalizarCaracteristicas(_X):
  return _X / 255

La función *definirCapaOculta* está diseñada para determinar la estructura de una capa oculta en una red neuronal.
- ce representa el número de características de entrada (input features) que la red neuronal recibirá.
- ne es el número de clases únicas o diferentes que hay en el conjunto de etiquetas y, lo cual es importante para la clasificación.
- co es el número calculado de neuronas en la capa oculta.

La fórmula utilizada es una heurística común que toma la raíz cuadrada del producto del número de características de entrada y el número de clases únicas. Esto es solo una estimación y el número óptimo de neuronas puede variar dependiendo del problema específico.
La razón detrás de este valor específico de epsilon_init está relacionada con la técnica de inicialización de Xavier, también conocida como "glorot initialization". Esta técnica se desarrolló para abordar el problema de la inicialización de los pesos en redes neuronales, donde los valores iniciales de los pesos pueden afectar significativamente la convergencia y el rendimiento del modelo durante el entrenamiento.
La función devuelve estos tres valores, que pueden ser utilizados para definir la arquitectura de una red neuronal.

In [5]:
def definirCapaOculta(X, y):
  ce = X.shape[1]
  ne = len(np.unique(y))
  co = int(np.sqrt(ce * ne))
  return ce, ne, co

La función *inicializarPesos* se utiliza para inicializar los pesos de una red neuronal antes del entrenamiento.
La inicialización de los pesos es un paso crucial en la configuración de una red neuronal. Una buena inicialización puede ayudar a mejorar la velocidad de convergencia durante el entrenamiento y evitar problemas como el desvanecimiento o la explosión de gradientes. La técnica utilizada aquí es una forma de inicialización basada en la regla de Xavier, que considera el tamaño de las capas anterior y siguiente para determinar el rango de los valores aleatorios.

In [6]:
def inicializarPesos(ce, ne, co):
  init_bound = np.sqrt(2.0 / (ce + co))
  Theta1 = np.random.uniform(-init_bound, init_bound, (co, ce + 1))
  Theta2 = np.random.uniform(-init_bound, init_bound, (ne, co + 1))
  return Theta1, Theta2


La función *sigmoide* es una implementación de la función de activación sigmoide.

La función sigmoide es una función matemática que tiene una característica forma de “S”. Se utiliza comúnmente como función de activación en redes neuronales porque tiene una salida que varía de 0 a 1, lo que la hace útil para problemas de clasificación binaria. Además, su derivada es fácil de calcular, lo que es útil durante el proceso de retropropagación en el entrenamiento de redes neuronales.

In [7]:
def sigmoide(z):
  z = np.clip(z, -500, 500)
  return 1.0 / (1.0 + np.exp(-z))

La función *sigmoideGradiente* calcula el gradiente de la función de activación sigmoide, que es útil durante el proceso de retropropagación en el entrenamiento de redes neuronales.

Este gradiente indica cómo cambia la salida de la función sigmoide con respecto a su entrada ( z ), y es crucial para ajustar los pesos de la red neuronal durante el aprendizaje.

In [8]:
def sigmoideGradiente(z):
  sig = sigmoide(z)
  return sig * (1 - sig)

La función *redneuronalFuncionCosto* es una implementación de la función de costo y el gradiente para una red neuronal en el contexto de aprendizaje supervisado.

- Se realiza un reshape de los parámetros de la red neuronal para obtener las matrices de pesos Theta1 y Theta2.
- Se calcula la activación de cada capa de la red utilizando la función sigmoide.
- Se transforma el vector de etiquetas y en una matriz binaria para la clasificación multiclase.
- Se añade un término de regularización para evitar el sobreajuste.
- Se calcula la función de costo con regularización.
- Se realiza la retropropagación para calcular los errores y gradientes de cada capa.
- Se ajustan los gradientes con un término de regularización y se devuelven junto con la función de costo.

In [9]:
def redneuronalFuncionCosto(rn_p, ce, co, ne, X, y, lambda_):
  Theta1 = np.reshape(rn_p[:co * (ce + 1)], (co, (ce + 1)))
  Theta2 = np.reshape(rn_p[(co * (ce + 1)):], (ne, (co + 1)))
  m = y.size
  J = 0
  Theta1_grad = np.zeros(Theta1.shape)
  Theta2_grad = np.zeros(Theta2.shape)
  a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
  a2 = sigmoide(a1.dot(Theta1.T))
  a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
  a3 = sigmoide(a2.dot(Theta2.T))
  y_matrix = y.values.reshape(-1)
  y_matrix = np.eye(ne)[y_matrix]
  temp1 = Theta1
  temp2 = Theta2
  reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))
  J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term
  delta_3 = a3 - y_matrix
  delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoideGradiente(a1.dot(Theta1.T))
  Delta1 = delta_2.T.dot(a1)
  Delta2 = delta_3.T.dot(a2)
  Theta1_grad = (1 / m) * Delta1
  Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
  Theta2_grad = (1 / m) * Delta2
  Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]
  grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
  return J, grad

La función *realizarPredicciones* se utiliza para hacer predicciones con una red neuronal ya entrenada.

- Se añade un término de sesgo a las entradas y a la primera capa oculta antes de calcular la activación.
- Se utiliza la función sigmoide para calcular la activación de las capas ocultas.
- Se utiliza np.argmax para seleccionar la clase con la mayor probabilidad como la predicción final para cada ejemplo.

In [10]:
def realizarPredicciones(X, Theta1, Theta2):
  m = X.shape[0]
  p = np.zeros(m)
  h1 = sigmoide(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
  h2 = sigmoide(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
  p = np.argmax(h2, axis=1)
  return p

La función *calcularPrecision* se utiliza para calcular la precisión de un conjunto de predicciones comparándolas con los valores reales.
- Se utiliza np.round para convertir las probabilidades de las predicciones en valores binarios (0 o 1).
- Se calcula la precisión como el promedio de las predicciones correctas (donde las predicciones redondeadas son iguales a las etiquetas reales y_test).
- Se imprime la precisión en formato de porcentaje con diez decimales de precisión.

In [11]:
def calcularPrecision(predicciones, y):
  predicciones_redondeadas = np.round(predicciones)
  precision = np.mean(predicciones_redondeadas == y_test) * 100
  print("Precisión de las predicciones en el conjunto de prueba: {:.10f}%".format(precision))

**(2) Implementación y entrenamiento de la Red Neuronal.**

Este fragmento de código está utilizando las funciones previamente definidas para cargar un conjunto de datos desde un archivo CSV y luego normalizar las características de los conjuntos de entrenamiento y prueba.

In [14]:
X_train, X_test, y_train, y_test = cargarDataset('/content/gdrive/MyDrive/SIS420/Laboratorio-N5_-_SIS420/Datasets/TMNIST Alphabet/94_character_TMNIST.csv')
x_train = normalizarCaracteristicas(X_train)
X_test = normalizarCaracteristicas(X_test)

Este fragmento de código está configurando una red neuronal, definiendo la capa oculta y los pesos iniciales.
1. Se llama a la función definirCapaOculta para determinar el número de características de entrada, el número de etiquetas (clases) y el tamaño de la capa oculta basándose en los datos de entrenamiento.
2. Luego, se utiliza la función inicializarPesos para crear dos matrices de pesos (Theta1 y Theta2) con valores iniciales aleatorios adecuados para la red neuronal.
3. Finalmente, se aplanan estas matrices en vectores unidimensionales y se concatenan para formar un solo vector llamado redneuronal_parametros. Este vector contiene todos los parámetros (pesos) de la red neuronal que serán ajustados durante el entrenamiento.

In [15]:
caracteristicas_entrada, numero_etiquetas, capa_oculta = definirCapaOculta(X_train, y_train)
Theta1, Theta2 = inicializarPesos(caracteristicas_entrada, numero_etiquetas, capa_oculta)
redneuronal_parametros = np.concatenate([Theta1.ravel(), Theta2.ravel()])

Este fragmento de código está utilizando una función de costo para entrenar una red neuronal y luego obtener los parámetros optimizados.
1. Se crea una función lambda que encapsula la función de costo de la red neuronal. Esta función se pasará al algoritmo de optimización.
2. Se llama a optimize.minimize con la función de costo, los parámetros iniciales de la red neuronal, y se especifican opciones adicionales para el proceso de optimización.
3. Después de la optimización, se reestructuran los parámetros optimizados en las matrices de pesos Theta1 y Theta2 que corresponden a las capas de la red neuronal.

In [16]:
funcionCosto = lambda p: redneuronalFuncionCosto(p, caracteristicas_entrada, capa_oculta, numero_etiquetas, X_train, y_train, 0.1)
redneuronal_parametros = optimize.minimize(funcionCosto, redneuronal_parametros, jac=True, method='L-BFGS-B', options={'disp': True, 'maxiter': 100}).x
Theta1 = np.reshape(redneuronal_parametros[:capa_oculta * (caracteristicas_entrada + 1)], (capa_oculta, (caracteristicas_entrada + 1)))
Theta2 = np.reshape(redneuronal_parametros[(capa_oculta * (caracteristicas_entrada + 1)):], (numero_etiquetas, (capa_oculta + 1)))

**(3)Cálculo de la presición.**

Se realizan las predicciones y el posterior cálculo de la precisión del modelo para verificar su efectividad.

In [21]:
calcularPrecision(realizarPredicciones(X_test, Theta1, Theta2), y_test)

Precisión de las predicciones en el conjunto de prueba: 80.3480545066%


*Nota: Se establecieron un nivel bajo de iteraciones, esto debido a que el optimize.minimize tarda al menos 4 minutos promedio en solamente 10 iteraciones, esto debido a que el tamaño del dataset es demasiado grande.*

# **Segunda Parte**

**(1) Declaración de las funciones**

La clase *RedNeuronalMLS* es una definición de una red neuronal simple utilizando PyTorch, una biblioteca de aprendizaje profundo.
* ce representa el número de características de entrada.
* ne es el número de etiquetas de salida (por ejemplo, clases en un problema de clasificación).
* co es el número de neuronas en la capa oculta, calculado como la raíz cuadrada del producto de ce y ne.
* nn.Linear es una capa que aplica una transformación lineal a los datos entrantes: ( y = xA^T + b ).
* F.sigmoid es una función de activación que transforma los valores de entrada en un rango entre 0 y 1.

Es importante mencionar que la función de activación F.sigmoid está en desuso en versiones más recientes de PyTorch, y se recomienda usar torch.sigmoid o F.relu en su lugar para mejores resultados y prácticas actualizadas.

In [12]:
class RedNeuronalMLS(nn.Module):
    def __init__(self, ce, ne):
        super(RedNeuronalMLS, self).__init__()
        co = int(np.sqrt(ce * ne))
        self.capa_oculta = nn.Linear(ce, co)
        self.capa_salida = nn.Linear(co, ne)
    def forward(self, x):
        x = self.capa_oculta(x)
        x = F.sigmoid(x)
        x = self.capa_salida(x)
        return x

La función *calcularPrecision* se utiliza para evaluar la precisión de un modelo de red neuronal en PyTorch.
* Se itera sobre un loader, que generalmente es un DataLoader de PyTorch que proporciona lotes de datos y etiquetas.
* Se utiliza modelo.eval() para poner el modelo en modo de evaluación, lo que es importante para algunas capas como dropout o batch normalization que tienen comportamientos diferentes durante el entrenamiento y la evaluación.
* Se utiliza torch.no_grad() para desactivar el cálculo de gradientes, ya que no es necesario durante la evaluación y ahorra memoria y tiempo de cómputo.
* Se calculan las predicciones y se compara con las etiquetas verdaderas y para determinar la cantidad de predicciones correctas.
* Finalmente, se devuelve la precisión calculada como un porcentaje y la lista de predicciones.

In [13]:
def calcularPrecisionModel(loader, modelo):
  num_correct = 0
  num_samples = 0
  modelo.eval()
  predicciones = []
  with torch.no_grad():
      for x, y in loader:
          x = x.to(device=dispositivo)
          y = y.to(device=dispositivo)
          x = x.reshape(x.shape[0], -1)
          scores = modelo(x)
          _, predictions = scores.max(1)
          predicciones.append(predictions)
          num_correct += (predictions == y).sum()
          num_samples += predictions.size(0)
  modelo.train()
  return num_correct/num_samples, predicciones

**(2) Implementación de la Red Neuronal.**

Este fragmento de código prepara los datos para ser utilizados en un entorno de PyTorch, seleccionando el dispositivo de cómputo y convirtiendo los conjuntos de datos en tensores y datasets de PyTorch.
* Se utiliza torch.device para seleccionar automáticamente CUDA si está disponible, lo que permite el uso de GPU para el entrenamiento, de lo contrario, se utiliza la CPU.
* Se llama a la función cargarDataset para obtener los conjuntos de datos divididos en entrenamiento y prueba.
* Se convierten los conjuntos de datos en tensores de PyTorch, que son estructuras de datos compatibles con las operaciones de PyTorch y se pueden transferir al dispositivo seleccionado (CPU o GPU).
* Se crean datasets de PyTorch (TensorDataset) que emparejan las entradas con sus etiquetas correspondientes. Estos datasets se pueden utilizar para cargar los datos en un DataLoader para iterar durante el entrenamiento y la evaluación.

In [14]:
dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, X_test, y_train, y_test = cargarDataset('/content/gdrive/MyDrive/SIS420/Laboratorio-N5_-_SIS420/Datasets/TMNIST Alphabet/94_character_TMNIST.csv')
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.int64)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

Este fragmento de código configura los parámetros para entrenar un modelo de red neuronal utilizando PyTorch.
* caracteristicas_entrada es el número de características (o atributos) que cada muestra de entrada tiene.
* numero_etiquetas es el número de clases diferentes que el modelo intentará predecir.
* tasa_aprendizaje es un hiperparámetro que controla cuánto se ajustan los pesos del modelo con respecto al gradiente del error en cada actualización.
* tamaño_lote y numero_iteraciones son hiperparámetros que controlan el tamaño de los lotes de datos procesados y el número de veces que se entrenará el modelo en todo el conjunto de datos, respectivamente.
* train_loader y test_loader son objetos de PyTorch que automatizan el proceso de iteración sobre los conjuntos de datos durante el entrenamiento y la evaluación, respectivamente.

In [15]:
caracteristicas_entrada = X_train.shape[1]
numero_etiquetas = len(np.unique(y_train))
tasa_aprendizaje = 0.001
tamaño_lote = 10000
numero_iteraciones = 9
train_loader = DataLoader(train_dataset, batch_size=tamaño_lote, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=tamaño_lote, shuffle=False)

Este fragmento de código establece y entrena un modelo de red neuronal utilizando la biblioteca PyTorch.
* Se inicializa el modelo de la red neuronal y se configura para usar el dispositivo adecuado (CPU o GPU).
* Se utiliza la entropía cruzada como función de pérdida, que es adecuada para tareas de clasificación multiclase.
* Se elige Adam como el optimizador, que ajustará los pesos del modelo para minimizar la función de pérdida.
* El bucle for externo itera sobre las epoch, y el bucle for interno itera sobre los lotes de datos proporcionados por train_loader.
* En cada iteración, se calculan las puntuaciones del modelo, se calcula la pérdida, se limpian los gradientes antiguos, se realiza la retropropagación para obtener nuevos gradientes, y finalmente se actualizan los pesos del modelo.

In [16]:
modelo = RedNeuronalMLS(caracteristicas_entrada, numero_etiquetas).to(dispositivo)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(modelo.parameters(), lr=tasa_aprendizaje)
for epoch in range(numero_iteraciones):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device=dispositivo)
        targets = targets.to(device=dispositivo)
        data = data.reshape(data.shape[0], -1)
        scores = modelo(data)
        loss = criterion(scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|██████████| 22/22 [00:22<00:00,  1.01s/it]


**(3) Cálculo de la presición.**

Este fragmento de código utiliza la función calcularPrecision para evaluar la precisión del modelo de red neuronal tanto en el conjunto de entrenamiento como en el de prueba.

In [17]:
p_train, pred_train = calcularPrecisionModel(train_loader, modelo)
p_test, pred_test = calcularPrecisionModel(test_loader, modelo)
print(f"Presición en el dataset de entrenamiento: {p_train*100:.2f}")
print(f"Presición en el dataset de prueba: {p_test*100:.2f}")

Presición en el dataset de entrenamiento: 85.56
Presición en el dataset de prueba: 84.89
